In [32]:
import time
import re
import pandas as pd
import numpy as np
from datetime import datetime

In [33]:
from bs4 import BeautifulSoup

# Load the html content from the txt file
file_path = r"C:\Users\henry\OneDrive - Louisiana State University\Desktop\Dota2Predictor\txt files\manual_webscraper_html.txt"

with open (file_path, 'r', encoding = 'utf-8') as file:
    file_content = file.read()

soup = BeautifulSoup(file_content, 'html.parser')


In [34]:
print(soup.prettify())

<html class="notranslate" lang="en" translate="no">
 <head>
  <link href="https://fonts.googleapis.com" rel="preconnect"/>
  <link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
  <link as="script" fetchpriority="low" href="/_next/static/chunks/webpack-37dc116c646d133e.js" rel="preload"/>
  <script async="" defer="" src="https://matomo.stratz.com/matomo.js" type="text/javascript">
  </script>
  <script async="" src="/_next/static/chunks/fd9d1056-e4ed69b157b2b858.js">
  </script>
  <script async="" src="/_next/static/chunks/3853-276c7084e681b78c.js">
  </script>
  <script async="" src="/_next/static/chunks/main-app-d25a7f1eba336e98.js">
  </script>
  <link as="script" href="https://polyfill.io/v3/polyfill.min.js?features=Intl.RelativeTimeFormat,Intl.RelativeTimeFormat.~locale.en,ResizeObserver" rel="preload"/>
  <link as="script" href="/dotaconstants?languageEnum=ENGLISH" rel="preload"/>
  <meta content="notranslate" name="google"/>
  <link href="/favicon.ico" rel="s

In [35]:
# Load previous data from csv file
df = pd.read_csv(r"C:\Users\henry\OneDrive - Louisiana State University\Desktop\Dota2Predictor\dota.csv")

In [36]:
# Get MatchID from title

title_tag = soup.find('title').text

matchid = int(title_tag.split(' ')[1])


In [37]:
print(matchid)

7343241517


In [38]:
# Determine which row to add match information

row_index = df.loc[df['MatchID'] == matchid].index         # Take the index of the row with the correct matchid
print(row_index)

Index([0], dtype='int64')


In [39]:
my_hero = df.loc[row_index,'Hero']
my_hero = my_hero.iloc[0].lower().replace(' ','_')

In [40]:
print(my_hero)

rubick


In [41]:
all_hero_tags = soup.find_all('img', class_ = 'sc-9ad1ed81-0 gWWkux')

radiant_team_tags = all_hero_tags[:5]
dire_team_tags = all_hero_tags[5:10]

In [42]:
radiant_team = []
dire_team = []

#Extract hero name from the image tag and append to enemey_heroes 
for i in radiant_team_tags:
     hero = i['src'].split('/')[6].replace('_horz.png','')
     radiant_team.append(hero)    

for i in dire_team_tags:
     hero = i['src'].split('/')[6].replace('_horz.png','')
     dire_team.append(hero)                           

In [43]:
print(dire_team)
print(radiant_team)

['bloodseeker', 'primal_beast', 'centaur', 'spirit_breaker', 'venomancer']
['skeleton_king', 'invoker', 'legion_commander', 'earthshaker', 'rubick']


In [44]:
if str(my_hero) in radiant_team:
    team = 'Radiant'
    df.loc[row_index, 'Team'] = team
    enemy_heroes = dire_team
    ally_heroes = radiant_team
else:
    team = 'Dire'
    df.loc[row_index, 'Team'] = team
    enemy_heroes = radiant_team
    ally_heroes = dire_team
    

In [45]:
position_number = 1
for enemy_hero, ally_hero in zip(enemy_heroes,ally_heroes) :
    df.loc[row_index,'Enemy_pos' + str(position_number)] = enemy_hero
    if position_number != 5:
        df.loc[row_index,'Ally_pos' + str(position_number)] = ally_hero
    position_number += 1


In [46]:
user_tag = soup.find('a', href = '/players/122262109')

In [47]:
if soup.find('div', class_ = "hitagi__sc-1ah81hi-0 jhafUR"):
    networth_str = user_tag.find_next('div', class_ = "hitagi__sc-1ah81hi-0 jhafUR").text.replace('k', "")
    networth = float(networth_str) * 1000

In [48]:
if soup.find_all('img', class_ = "sc-fddce6ee-1 fRsjAO"):
    for img_tag in soup.find_all('img', class_ = "sc-fddce6ee-1 fRsjAO"):
        if my_hero in img_tag.get('src'):
            my_hero_tag = img_tag
    
    networth = my_hero_tag.find_next('span', class_ = "hitagi__sc-6oal1n-0 kQjyLc").text.replace(',' , "")

In [49]:
# Extract Date and add to row
date_time = soup.find_all('span', class_ = "hitagi__sc-t22b45-1 gvSdww")[10].find('span').text

date_time = datetime.strptime(date_time, '%b %d, %Y, %I:%M %p')

df.loc[row_index, 'Date'] = date_time

In [50]:
# Add networth to row
df.loc[row_index,'Networth'] = networth

In [51]:
df.iloc[0]

MatchID                    7343241517
Date              2023-09-19 23:29:00
Hero                           rubick
Match Duration                  36:41
Result                           Lost
Kills                               9
Deaths                              9
Assists                            12
Networth                      11200.0
Team                          Radiant
Enemy_pos1                bloodseeker
Enemy_pos2               primal_beast
Enemy_pos3                    centaur
Enemy_pos4             spirit_breaker
Enemy_pos5                 venomancer
Ally_pos1               skeleton_king
Ally_pos2                     invoker
Ally_pos3            legion_commander
Ally_pos4                 earthshaker
Name: 0, dtype: object

In [52]:
csv_dota2 = 'dota.csv'
df.to_csv(csv_dota2, index=False)